In [7]:
# import packages 
import sys
sys.path.append('Auxiliary/')
import xarray as xr 
import rioxarray as rio 
import os
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
from pathlib import Path
from rocksdbutils_copy import *
import cv2
import albumentations as A
from math import ceil as mceil
from helper import *

    

img_size = 128
## create metadata file
metadata = {
    'inputs': {
        'inputs_shape': (4, 6, img_size, img_size),  
        'inputs_dtype': np.float32     
    },
    'labels': {
        'labels_shape': (4, img_size, img_size),          
        'labels_dtype': np.float32
    }
}

## define function to load imgs and labs
def names2array_function(names):

    variables2use=['B2','B3','B4','B8']#,'NDVI']

    image_path, label_path = names
    # load image
    img = xr.open_dataset(image_path)
    image = np.concatenate([img[var].values[None] for var in variables2use],0)
    
    # load label
    ds = xr.open_dataset(label_path)
    label = np.asarray(ds['band_data'].values)
    label[np.isnan(label)] = 0
    return [image, label] 

#def vrt2array_function(vrt):



In [ ]:
## create list of images and labels
dataFolder = '../../../../data/fields/'
imgvrt = dataFolder + 'vector/Force_X_from_68_to_69_Y_from_42_to_42.vrt'

labs = [getFilelist('/home/ai4boundaries/sentinel2/masks/' + folder, '.tif') for folder in folders]
labs = [lab for list in labs for lab in list]
print(len(imgs), len(labs))
imgs.sort()
labs.sort()

aa = checkemptyNC(labs)
# exclude images with empty labels and their labels
labs = [lab for i, lab in enumerate(labs) if i not in aa]
imgs = [img for i, img in enumerate(imgs) if i not in aa]

print(len(imgs), len(labs))

In [ ]:
img_lab_paths = []
for i in range(len(imgs)):
    img_lab_paths.append((imgs[i], labs[i]))

# img_lab_paths = img_lab_paths[:2]


# aa = names2array_function(img_lab_paths[4])[0]
# aa.astype(np.int16)

## create db
output_dir = '/home/output/rocks_db/All_but_LU_transformed.db'
os.makedirs(output_dir, exist_ok=True)

rasters2rocks = Rasters2RocksDB(
    lstOfTuplesNames=img_lab_paths,            
    names2raster_function=names2array_function,  
    metadata=metadata,                       
    flname_prefix_save=output_dir,           
    batch_size=2,
    transformT=TrainingTransform_for_rocks_Train(),
    transformV=TrainingTransform_for_rocks_Valid(),
    #stride_divisor=2,                           
    #train_split=0.8,                         
    Filter=img_size,
    split_type='sequential'                  
)

rasters2rocks.create_dataset()